In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset

In [ ]:
ds = load_dataset("stanfordnlp/imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from nltk.tokenize import word_tokenize
from collections import defaultdict
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from tqdm import tqdm

In [ ]:
train = ds['train'].to_pandas()
test = ds['test'].to_pandas()
print(train.shape, test.shape)

texts = train['text'].values
labels = train['label'].values
val_texts = test['text'].values
val_labels = test['label'].values

(25000, 2) (25000, 2)


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):

    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word.isalpha()]

    return tokens

train_clean = list(map(clean_text, texts))
val_clean = list(map(clean_text, val_texts))

max_seq_len = 512
train_clean = [sentence[:max_seq_len] for sentence in train_clean]
val_clean = [sentence[:max_seq_len] for sentence in val_clean]

vocab = defaultdict(lambda: len(vocab)+1)
tokenized_texts_idx = [[vocab[word] for word in sentence] for sentence in train_clean]
vocab.default_factory = None
vocab_size = len(vocab)
val_tokenized_texts_idx = [[vocab[word] if word in vocab else 0 for word in sentence] for sentence in val_clean]



tokenized_texts_idx = [torch.tensor(sentence) for sentence in tokenized_texts_idx]
val_tokenized_texts_idx = [torch.tensor(sentence) for sentence in val_tokenized_texts_idx]

padded_sequences = pad_sequence(tokenized_texts_idx, batch_first=True, padding_value=0)
val_padded_sequences = pad_sequence(val_tokenized_texts_idx, batch_first=True, padding_value=0)  # Padding with 0

labels = torch.tensor(labels)
val_labels = torch.tensor(val_labels)

<ipython-input-39-83ca714669fa>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)
<ipython-input-39-83ca714669fa>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_labels = torch.tensor(val_labels)


In [ ]:
train_dataset = TensorDataset(padded_sequences, labels)
val_dataset = TensorDataset(val_padded_sequences, val_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
def trainloop(model, train_loader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    total = 0

    for texts, labels in tqdm(train_loader):
        texts, labels = texts.to(device), labels.to(device)

        optimizer.zero_grad()
        predictions = model(texts)

        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(predictions, 1)
        correct = (predicted == labels).sum().item()

        epoch_loss += loss.item() * texts.size(0)
        epoch_acc += correct
        total += texts.size(0)

    return epoch_loss / total, epoch_acc / total


In [ ]:
def evaluate(model, val_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    total = 0

    with torch.no_grad():
        for texts, labels in tqdm(val_loader):
            texts, labels = texts.to(device), labels.to(device)

            predictions = model(texts)
            loss = criterion(predictions, labels)

            _, predicted = torch.max(predictions, 1)
            correct = (predicted == labels).sum().item()

            epoch_loss += loss.item() * texts.size(0)
            epoch_acc += correct
            total += texts.size(0)

    return epoch_loss / total, epoch_acc / total

In [ ]:
class TransformerTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads, hidden_dim, num_layers, output_dim, max_len=512, dropout=0.5):
        super(TransformerTextClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.positional_encoding = nn.Parameter(self._generate_positional_encoding(max_len, embedding_dim), requires_grad=False)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(embedding_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def _generate_positional_encoding(self, max_len, embedding_dim):

        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / embedding_dim))
        pe = torch.zeros(max_len, embedding_dim)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)

    def forward(self, x):
        seq_len = x.size(1)

        embedded = self.embedding(x)
        embedded = embedded + self.positional_encoding[:, :seq_len, :]

        embedded = self.dropout(embedded)

        transformer_out = self.transformer_encoder(embedded)  # (batch_size, seq_length, embedding_dim)

        pooled = transformer_out.mean(dim=1)  # (batch_size, embedding_dim)


        pooled = self.dropout(pooled)

        output = self.fc(pooled)

        return output

In [ ]:
embedding_dim = 128
num_heads = 8
hidden_dim = 256
num_layers = 3
output_dim = 2
max_len = 512
dropout = 0.5


model = TransformerTextClassifier(
    vocab_size=vocab_size+1,
    embedding_dim=embedding_dim,
    num_heads=num_heads,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    output_dim=output_dim,
    max_len=max_len,
    dropout=dropout
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
model.to(device)
num_epochs = 5
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')

    train_loss, train_acc = trainloop(model, train_loader, criterion, optimizer, device)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')

    val_loss, val_acc = evaluate(model, val_dataloader, criterion, device)
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

Epoch 1/5


100%|██████████| 782/782 [00:14<00:00, 53.77it/s]


Train Loss: 0.6306, Train Acc: 0.6236


100%|██████████| 782/782 [00:04<00:00, 194.65it/s]


Val Loss: 0.5395, Val Acc: 0.7225
Epoch 2/5


...

Val Loss: 0.5716, Val Acc: 0.8001
Epoch 5/5


100%|██████████| 782/782 [00:14<00:00, 54.09it/s]


Train Loss: 0.4552, Train Acc: 0.7863


100%|██████████| 782/782 [00:04<00:00, 192.68it/s]

Val Loss: 0.6000, Val Acc: 0.8048
